In [1]:
import numpy as np
import pandas as pd
import pymystem3
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

In [44]:
train_df = pd.read_excel('Data/train_data_exam_1.xlsx')
test_df = pd.read_excel('Data/test_data_exam_1.xlsx')

# deleting nan and rows containing non-string value at 'original_data'
train_df = train_df.dropna()
train_df = train_df[pd.to_numeric(train_df['original_data'], errors='coerce').isna()]
test_df = test_df.dropna()
test_df = test_df[pd.to_numeric(test_df['original_data'], errors='coerce').isna()]

X_train, y_train = train_df['original_data'], train_df['score']
X_test, y_test = test_df['original_data'], test_df['score']

In [4]:
m = pymystem3.Mystem()


def lemmatize(X):
    data = []
    for s in tqdm(X):
        s = re.sub(r"[,.?“/!@#$1234567890#—_ツ►๑۩۞۩•*”˜˜”*°°*`)(]", ' ', s)
        s = m.lemmatize(s)
        s = ''.join(s)
        data.append(s)
    return data

In [ ]:
X_train = lemmatize(X_train)
X_test = lemmatize(X_test)

 75%|███████▍  | 13961/18715 [00:59<00:19, 242.90it/s]

In [63]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1), stop_words=None, max_df=0.1, min_df=0.001)

X_train = tfidf_vectorizer.fit_transform(X_train).toarray()
train_features = tfidf_vectorizer.get_feature_names_out()

X_test = tfidf_vectorizer.fit_transform(X_test).toarray()
test_features = tfidf_vectorizer.get_feature_names_out()

In [ ]:
def prepare_test_data(train_features, test_features, X_test):
    dct = {word: idx for idx, word in enumerate(train_features)}
    prepared_X_test = np.empty((X_test.shape[0], train_features.size))

    for i, feature in enumerate(test_features):
        if feature in dct:
            prepared_X_test[:, dct[feature]] = X_test[:, i]

    return prepared_X_test

prepare_test_data(train_features, test_features, X_test).shape

In [ ]:
np.save('Data/X_train', X_train)
np.save('Data/y_train', y_train)
np.save('Data/X_test', X_test)
np.save('Data/y_test', y_test)